# Open OnDemandのインストール

## 前提

このnotebookでは、Slurmがインストールされた環境にOpen OnDemandをインストールします。
* Open OnDemandをインストールする対象は、OCS templatesの[OpenHPC-v2](https://github.com/nii-gakunin-cloud/ocs-templates/tree/master/OpenHPC-v2)に従って構築されたOpenHPCの環境とする。
* このnotebookでは、上記のOpenHPC-v2テンプレートの実行の過程で作成された`group_vars/*.yml`およびansibleの設定ファイルを再利用する。

## 概要

Open OnDemandは、基本的にはログインノードにインストールする前提で設計されています。このため、OCS templatesのOpenHPC-v2に従って構築された環境では、マスターノードにOpen OnDemandをインストールします。

![構成](./images/ohpc+ood.svg)

## 準備

以下を実行して、OpenHPC環境を作成したnotebookのディレクトリ(`group_vars`や`inventory.yml`のあるディレクトリ)をチェックします。

In [ ]:
!ls -d ../OpenHPC-v2/work/group_vars \
    ../OpenHPC-v2/group_vars \
    ../../templates/OpenHPC-v2/group_vars \
    ../../templates/OpenHPC-v2/work/group_vars | \
    sed -e 's,/group_vars,,'

`ohpc_dir`に、`group_vars`や`inventory.yml`があるディレクトリを指定します。

In [ ]:
ohpc_dir = '../../templates/OpenHPC-v2/work'

上記で指定したディレクトリから必要なファイルをコピーします。

In [ ]:
!cp -pr {ohpc_dir}/group_vars \
    {ohpc_dir}/ansible.cfg \
    {ohpc_dir}/inventory.yml .

OpenHPC-v2テンプレートで作成したgroup_varsの読み込みのため、指定したユニットグループ名を調べます。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

`ugroup_name`にユニットグループ名を指定します。

In [ ]:
ugroup_name = 

OpenHPC-v2環境構築時のgroup_varsを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

パラメータの内容を表示して確認します。

In [ ]:
import json
print(json.dumps(gvars, indent=2))

ansibleで各ノードにアクセスできる状態であることを確認します。

In [ ]:
!ansible -m ping {ugroup_name}

## パッケージのインストール

以下の4個のセルを実行することにより、Open OnDemandのインストールに必要となるパッケージリポジトリを有効化・追加します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'dnf -y config-manager --set-enabled powertools'

In [ ]:
!ansible {ugroup_name}_master -b -m dnf -a 'name=epel-release state=present'

In [ ]:
!ansible {ugroup_name}_master -b -a 'dnf -y module enable ruby:2.7 nodejs:14'

In [ ]:
!ansible {ugroup_name}_master -b -m dnf -a \
    'name=https://yum.osc.edu/ondemand/2.0/ondemand-release-web-2.0-1.noarch.rpm \
    disable_gpg_check=true state=present'

以下のセルを実行することで、Open OnDemandのパッケージとそれに依存するパッケージをインストールします。

In [ ]:
!ansible {ugroup_name}_master -b -m dnf -a 'name=ondemand state=present'

Open OnDemandからのシェルアクセスのために必要となる、websockifyをインストールします。

In [ ]:
!ansible {ugroup_name}_master -b -m dnf -a 'name=python3-websockify state=present'